# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 


- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [31]:
import math
import re
import heapq
from collections import Counter, defaultdict
import math
import re
import heapq
from collections import Counter, defaultdict

class SpellCorrector:
    def __init__(self, corpus_path="big.txt", fivegram_path="fivegrams.txt"):
        self.unigrams = Counter()
        self.bigrams = defaultdict(Counter)
        self.total_words = 0
        self._load_corpus(corpus_path)
        self.fivegrams = {}
        self.fivegram_prefix = defaultdict(int)
        self._load_fivegrams(fivegram_path)

    def _load_corpus(self, path):
        with open(path, "r", encoding="utf-8") as f:
            text = f.read().lower()
        words = re.findall(r"[a-z]+", text)
        self.unigrams = Counter(words)
        self.total_words = sum(self.unigrams.values())
        for i in range(len(words)-1):
            self.bigrams[words[i]][words[i+1]] += 1

    def _load_fivegrams(self, path):
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 6:
                    continue 
                try:
                    count = int(parts[5])
                except ValueError:
                    continue
                fivegram = tuple(parts[:5])
                self.fivegrams[fivegram] = count
                prefix = (fivegram[0], fivegram[1], fivegram[3], fivegram[4])
                self.fivegram_prefix[prefix] += count

    def edits1(self, word):
        letters = "abcdefghijklmnopqrstuvwxyz"
        splits = [(word[:i], word[i:]) for i in range(len(word)+1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word):
        return {e2 for e1 in self.edits1(word) for e2 in self.edits1(e1)}

    def known(self, words):
        return set(w for w in words if w in self.unigrams)

    def candidates(self, word):
        if word in self.unigrams:
            return {word}
        cand = self.known(self.edits1(word))
        if cand:
            return cand
        cand = self.known(self.edits2(word))
        if cand:
            return cand
        return {word}

    def edit_distance(self, a, b):
        m, n = len(a), len(b)
        dp = [[0]*(n+1) for _ in range(m+1)]
        for i in range(m+1):
            dp[i][0] = i
        for j in range(n+1):
            dp[0][j] = j
        for i in range(1, m+1):
            for j in range(1, n+1):
                if a[i-1] == b[j-1]:
                    dp[i][j] = dp[i-1][j-1]
                else:
                    dp[i][j] = 1 + min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1])
        return dp[m][n]

    def score_candidate(self, orig_word, candidate, left_context, right_context, penalty_weight=1.0):
        score = 0.0
        uni_prob = (self.unigrams[candidate] + 1) / (self.total_words + len(self.unigrams))
        score += math.log(uni_prob)
        
        if left_context:
            left = left_context[-1]
            bi_count = self.bigrams[left][candidate]
            total_left = sum(self.bigrams[left].values())
            bi_prob = (bi_count + 1) / (total_left + len(self.unigrams))
            score += math.log(bi_prob)
        
        if right_context:
            right = right_context[0]
            bi_count = self.bigrams[candidate][right]
            total_candidate = sum(self.bigrams[candidate].values())
            ri_prob = (bi_count + 1) / (total_candidate + len(self.unigrams))
            score += math.log(ri_prob)
        
        if len(left_context) >= 2 and len(right_context) >= 2:
            left2, left1 = left_context[-2], left_context[-1]
            right1, right2 = right_context[0], right_context[1]
            key = (left2, left1, candidate, right1, right2)
            count_5g = self.fivegrams.get(key, 0)
            prefix = (left2, left1, right1, right2)
            total_prefix = self.fivegram_prefix.get(prefix, 0)
            five_prob = (count_5g + 1) / (total_prefix + len(self.unigrams))
            score += math.log(five_prob)
        
        ed = self.edit_distance(orig_word, candidate)
        score -= penalty_weight * ed
        
        return score

    def correct_word(self, word, left_context=[], right_context=[]):
        best_candidate = word
        best_score = -float('inf')
        for cand in self.candidates(word):
            sc = self.score_candidate(word, cand, left_context, right_context)
            if sc > best_score:
                best_score = sc
                best_candidate = cand
        return best_candidate

    def correct_sentence(self, sentence, beam_width=5):
        tokens = sentence.split()
        beam = [(0.0, tokens)]
        for i in range(len(tokens)):
            new_beam = []
            for cumulative_score, seq in beam:
                word = seq[i]
                for cand in self.candidates(word):
                    new_seq = list(seq)
                    new_seq[i] = cand
                    new_score = 0.0
                    for j, w in enumerate(new_seq):
                        left = new_seq[:j]
                        right = new_seq[j+1:]
                        new_score += self.score_candidate(tokens[j], w, left, right)
                    heapq.heappush(new_beam, (new_score, new_seq))
            beam = heapq.nlargest(beam_width, new_beam, key=lambda x: x[0])
        best_score, best_seq = max(beam, key=lambda x: x[0])
        return " ".join(best_seq)




if __name__ == "__main__":
    corpus_file = "big.txt"         
    fivegram_file = "fivegrams.txt"

    corrector = SpellCorrector(corpus_path=corpus_file, fivegram_path=fivegram_file)



## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

## N-gram Dataset Selection
We use Norvig's "big.txt" because it is a widely available, large, and diverse corpus. Its broad vocabulary and frequency information enable robust probability estimation for unigrams and, indirectly, for candidate generation.

## Candidate Generation via Edits
 We generate candidates using one-edit (edits1) and, if needed, two-edits (edits2). This approach follows empirical evidence that most spelling mistakes are within one or two edit distances, and it relies on corpus frequencies to implicitly weight the candidates.

## Edit Distance Penalty
An edit distance penalty is applied to discourage corrections that require significant changes. This encourages minimal adjustments and balances the frequency-based probability with the similarity to the original word.

## Beam Search for Sentence Correction
A beam width of 5 is used for sentence-level correction to efficiently explore multiple candidate sequences while keeping computational costs manageable. This provides a balance between context sensitivity and runtime performance.

## Dataset Parsing for Evaluation
We parse error-marked sentences (e.g., <ERR targ=sister> siter </ERR>) to generate gold and noisy versions. This structured format facilitates accurate evaluation of the correction performance on realistic error data.

## Speed Evaluation
Measuring total and per-sentence correction times using Python’s time module ensures that our implementation is not only accurate but also efficient for practical applications.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

In [33]:
import re
import time
from collections import Counter


class NorvigCorrector:
    def __init__(self, corpus_path="big.txt"):
        with open(corpus_path, "r", encoding="utf-8") as f:
            text = f.read().lower()
        self.WORDS = Counter(re.findall(r'\w+', text))
        self.N = sum(self.WORDS.values())
    
    def P(self, word):
        return self.WORDS[word] / self.N
    
    def correction(self, word):
        return max(self.candidates(word), key=self.P)
    
    def candidates(self, word):
        return (self.known([word]) or 
                self.known(self.edits1(word)) or 
                self.known(set(self.edits2(word))) or 
                {word})
    
    def known(self, words):
        return set(w for w in words if w in self.WORDS)
    
    def edits1(self, word):
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)
    
    def edits2(self, word):
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))
    
    def correct_sentence(self, sentence):
        return " ".join(self.correction(word) for word in sentence.split())
    
    def norvig_correct_sentence(self, sentence):
        return self.correct_sentence(sentence)


def parse_error_sentence(sentence):
    """
    Parses a sentence with error markup of the form:
       <ERR targ=correct_word> error_word </ERR>
    """
    pattern = r"<ERR\s+targ=['\"]?([^\"'\s>]+)['\"]?>(.*?)</ERR>"
    gold_sentence = re.sub(pattern, r"\1", sentence)
    noisy_sentence = re.sub(pattern, r"\2", sentence)
    return gold_sentence, noisy_sentence

def load_dataset_from_file(file_path):
    gold_sentences = []
    noisy_sentences = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            gold, noisy = parse_error_sentence(line)
            gold_sentences.append(gold)
            noisy_sentences.append(noisy)
    return gold_sentences, noisy_sentences


def compute_word_accuracy(gold_sentence, corrected_sentence):
    gold_tokens = gold_sentence.split()
    corrected_tokens = corrected_sentence.split()
    correct = sum(1 for g, c in zip(gold_tokens, corrected_tokens) if g == c)
    return correct, len(gold_tokens)

def evaluate_corrector(gold_sentences, noisy_sentences, corrector):
    total_gold_words = 0
    total_correct = 0
    total_time = 0.0
    detailed_results = []
    
    for gold, noisy in zip(gold_sentences, noisy_sentences):
        start = time.time()
        corrected = corrector.correct_sentence(noisy)
        elapsed = time.time() - start
        total_time += elapsed
        
        correct_count, total = compute_word_accuracy(gold, corrected)
        total_correct += correct_count
        total_gold_words += total
        
        detailed_results.append({
            "gold": gold,
            "noisy": noisy,
            "corrected": corrected,
            "accuracy": correct_count / total
        })
    
    overall_accuracy = (total_correct / total_gold_words) * 100
    avg_time = total_time / len(gold_sentences)
    return overall_accuracy, total_time, avg_time, detailed_results


dataset_file = "eval.txt"
gold_sentences, noisy_sentences = load_dataset_from_file(dataset_file)

corpus_file = "big.txt"
fivegram_file = "fivegrams.txt"

improved_corrector = SpellCorrector(corpus_path=corpus_file, fivegram_path=fivegram_file)
baseline_corrector = NorvigCorrector(corpus_path=corpus_file)

imp_accuracy, imp_total_time, imp_avg_time, imp_details = evaluate_corrector(gold_sentences, noisy_sentences, improved_corrector)

base_accuracy, base_total_time, base_avg_time, base_details = evaluate_corrector(gold_sentences, noisy_sentences, baseline_corrector)

print("=== Corrector Evaluation ===")
print("Overall Word Accuracy: {:.2f}%".format(imp_accuracy))
print("Total Correction Time: {:.4f} sec".format(imp_total_time))
print("Average Correction Time per Sentence: {:.4f} sec".format(imp_avg_time))
print("\n=== Baseline (Norvig) Corrector Evaluation ===")
print("Overall Word Accuracy: {:.2f}%".format(base_accuracy))
print("Total Correction Time: {:.4f} sec".format(base_total_time))
print("Average Correction Time per Sentence: {:.4f} sec".format(base_avg_time))



=== Corrector Evaluation ===
Overall Word Accuracy: 70.00%
Total Correction Time: 174.8377 sec
Average Correction Time per Sentence: 0.5677 sec

=== Baseline (Norvig) Corrector Evaluation ===
Overall Word Accuracy: 70.24%
Total Correction Time: 26.7063 sec
Average Correction Time per Sentence: 0.0867 sec


#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)